In [2]:
import os
import pickle

parent_directory = os.path.abspath(os.curdir).rsplit('\\', 1)[0]
print('Parent Directory:',parent_directory)

# model_dtree = pickle.load(open('model_dtree.pkl', 'rb'))
# scaler = pickle.load(open('scaler.pkl', 'rb'))
# outlier_limits = pickle.load(open('outlier_limits.pkl', 'rb'))

outlier_limits = pickle.load(open(parent_directory + "\\models\\outlier_limits.pkl",'rb'))
scaler = pickle.load(open(parent_directory + "\\models\\scaler.pkl",'rb'))
model_dtree = pickle.load(open(parent_directory + "\\models\\model_dtree.pkl",'rb'))
# model_ann = pickle.load(open(parent_directory + "\\models\\model_ann.pkl",'rb'))

Parent Directory: c:\Users\barto\Documents\Lambton\Term 3\2023F-T3 AML 3104 - Neural Networks and Deep Learning\AML-3104-Project


In [3]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from flask import Flask,request,jsonify,render_template

In [102]:
        # age = float(request.form.get('58'))
        # balance = float(request.form.get('500'))
        # pdays = float(request.form.get('30'))
        # previous = float(request.form.get('2'))
        # job_retired_student = request.form.get('retired')
        # marital_single = request.form.get('single')
        # education_primary_secondary = request.form.get('primary')
        # housing_yes = request.form.get('yes')
        # loan_yes = request.form.get('yes')
        # contact_cellular_telephone = request.form.get('cellular')
        # month_feb_mar_apr_sep_oct_dec = request.form.get('feb')
        # poutcome_success = request.form.get('success')
        
age = float('1')
balance = float('0')
pdays = float('0')
previous = float('0')
job = 'unknown'
marital = 'divorced'
education = 'unknown'
housing = 'no'
loan = 'no'
contact = 'unknown'
month = 'jan'
poutcome = 'failure'
        

In [103]:
        model_data = pd.DataFrame({
                'age' : [age],
                'balance' : [balance],
                'pdays' : [pdays],
                'previous' : [previous],
                'job' : [job],
                'marital' : [marital],
                'education' : [education],
                'housing' : [housing],
                'loan' : [loan],
                'contact' : [contact],
                'month' : [month],
                'poutcome' : [poutcome],
                
                'default' : ['x'],
                                
                # 'age',
                # 'balance',
                # 'pdays'
                # 'previous',
                # 'job_retired_student',
                # 'marital_single',
                # 'education_primary_secondary',
                # 'default_yes',
                # 'housing_yes',
                # 'loan_yes',
                # 'contact_cellular_telephone',
                # 'month_feb_mar_apr_sep_oct_dec',
                # 'poutcome_success'
                                  
                })
model_data

,age,balance,pdays,previous,job,marital,education,housing,loan,contact,month,poutcome,default
0,1.0,0.0,0.0,0.0,unknown,divorced,unknown,no,no,unknown,jan,failure,x


In [104]:
        model_data['job_retired_student'] = model_data['job'].apply(lambda value: 1 if value.lower() in ['retired', 'student'] else 0)
        model_data['marital_single'] = model_data['marital'].apply(lambda value: 1 if value.lower() == 'single' else 0)
        model_data['education_primary_secondary'] = model_data['education'].apply(lambda value: 1 if value.lower() in ['primary','secondary'] else 0)
        model_data['default_yes'] = model_data['default'].apply(lambda value: 1 if value.lower() == 'yes' else 0)
        model_data['housing_yes'] = model_data['housing'].apply(lambda value: 1 if value.lower() == 'yes' else 0)
        model_data['loan_yes'] = model_data['loan'].apply(lambda value: 1 if value.lower() == 'yes' else 0)
        model_data['contact_cellular_telephone'] = model_data['contact'].apply(lambda value: 1 if value.lower() in ['cellular','telephone'] else 0)
        model_data['month_feb_mar_apr_sep_oct_dec'] = model_data['month'].apply(lambda value: 1 if value.lower() in ['feb', 'mar', 'apr', 'oct', 'sep', 'dec'] else 0)
        model_data['poutcome_success'] = model_data['poutcome'].apply(lambda value: 1 if value.lower() == 'success' else 0)
        
        model_data

,age,balance,pdays,previous,job,marital,education,housing,loan,contact,...,default,job_retired_student,marital_single,education_primary_secondary,default_yes,housing_yes,loan_yes,contact_cellular_telephone,month_feb_mar_apr_sep_oct_dec,poutcome_success
0,1.0,0.0,0.0,0.0,unknown,divorced,unknown,no,no,unknown,...,x,0,0,0,0,0,0,0,0,0


In [105]:
        x_test = model_data[['age',
                        'balance',
                        'pdays',
                        'previous',
                        'job_retired_student',
                        'marital_single',
                        'education_primary_secondary',
                        'default_yes', # Added
                        'housing_yes',
                        'loan_yes',
                        'contact_cellular_telephone',
                        'month_feb_mar_apr_sep_oct_dec',
                        'poutcome_success',
                        ]]
        x_test

,age,balance,pdays,previous,job_retired_student,marital_single,education_primary_secondary,default_yes,housing_yes,loan_yes,contact_cellular_telephone,month_feb_mar_apr_sep_oct_dec,poutcome_success
0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0


In [106]:
# Function to check for outliers and cap values of outliers
def RemoveOutliers(inData,inOutliers):
    
    # Initialize blank dataframes
    outData = inData
    
    # Loop through all features
    for colName in inData.columns:
    
        if colName in list(inOutliers.keys()):
        
            min_val = inOutliers.get(colName)[0]  # Lower limit
            max_val = inOutliers.get(colName)[1]  # Upper limit
        
        # If the value is lower/greater than the min/max limits, it is an outlier
            outliers = inData[(inData[colName] < min_val) | (inData[colName] > max_val)]
        
        # For outliers, apply capping -- replace the value with either the lower or upper limit
            outData[colName] = np.where( inData[colName] > max_val, max_val, np.where( inData[colName] < min_val, min_val, inData[colName] ) )
        
        # Summary per feature
            # print(f'No. of outliers for {colName}: {outliers.shape[0]} \t\tLimits: {round(min_val,4)} , {round(max_val,4)}')
        
        else:
            outData[colName] = inData[colName].values
    
    return outData
 # Check and handle outliers of the data. For the test set, the lower/upper limits will be the same as the training set.
x_test_no_outliers = RemoveOutliers(x_test,outlier_limits)
x_test_no_outliers

C:\Users\barto\AppData\Local\Temp\ipykernel_15060\1909324914.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outData[colName] = np.where( inData[colName] > max_val, max_val, np.where( inData[colName] < min_val, min_val, inData[colName] ) )
C:\Users\barto\AppData\Local\Temp\ipykernel_15060\1909324914.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outData[colName] = inData[colName].values


,age,balance,pdays,previous,job_retired_student,marital_single,education_primary_secondary,default_yes,housing_yes,loan_yes,contact_cellular_telephone,month_feb_mar_apr_sep_oct_dec,poutcome_success
0,10.5,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0


In [107]:
# Scale features
x_test_sc = scaler.transform(x_test_no_outliers)

In [108]:
# Predict target variable
y_pred_test_dtree = model_dtree.predict(x_test_sc)
y_pred_test_dtree

array([0], dtype=int64)

In [109]:
if [y_pred_test_dtree][0][0] == 1:
    result = 'Yes'
else:
    result = 'No'
result

'No'

In [110]:
print('hey')
y_pred_test_dtree

hey


array([0], dtype=int64)